In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
# ✅ Define file paths
csv_path = "E:/capstone_data_engeneering/code/flipkart_clean_df.csv"
db_path = "E:/capstone_data_engeneering/code/android_iphone.db"

In [3]:
# ✅ Ensure directory exists
os.makedirs(os.path.dirname(db_path), exist_ok=True)

In [4]:
# ✅ Load CSV into DataFrame
df = pd.read_csv(csv_path)

In [5]:
# ✅ Remove duplicates based on product name and brand
df = df.drop_duplicates(subset=["Product Name", "brand"])

In [6]:
# ✅ Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [7]:
# ✅ Drop existing tables (if any)
cursor.execute("DROP TABLE IF EXISTS brands;")
cursor.execute("DROP TABLE IF EXISTS products;")
cursor.execute("DROP TABLE IF EXISTS specifications;")
cursor.execute("DROP TABLE IF EXISTS ratings_reviews;")

In [8]:
# ✅ Create Main Table (brands)
cursor.execute("""
CREATE TABLE brands (
    brand_id INTEGER PRIMARY KEY AUTOINCREMENT,
    brand TEXT UNIQUE
);
""")

In [9]:
# ✅ Create Products Table (Linked to brands)
cursor.execute("""
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    brand_id INTEGER,
    product_name TEXT
);
""")

In [10]:
# ✅ Create Specifications Table (Linked to products)
cursor.execute("""
CREATE TABLE specifications (
    product_id INTEGER PRIMARY KEY,
    battery TEXT,
    display TEXT,
    camera TEXT,
    processor TEXT,
    ram REAL,
    rom REAL,
    type TEXT,
    FOREIGN KEY (product_id) REFERENCES products(product_id) ON DELETE CASCADE
);
""")

In [11]:
# ✅ Create Ratings & Reviews Table (Linked to products)
cursor.execute("""
CREATE TABLE ratings_reviews (
    product_id INTEGER PRIMARY KEY,
    ratings REAL,
    reviews REAL,
    FOREIGN KEY (product_id) REFERENCES products(product_id) ON DELETE CASCADE
);
""")

In [12]:
# ✅ Create Ratings & Reviews Table (Linked to products)
cursor.execute("""
CREATE TABLE price (
    product_id INTEGER PRIMARY KEY,
    price REAL,
    discount REAL,
    actual_price REAL,           
    FOREIGN KEY (product_id) REFERENCES products(product_id) ON DELETE CASCADE
);
""")

OperationalError: table price already exists

In [13]:
# ✅ Insert Data into brands table (Get unique brands)
for brand in df["brand"].unique():
    cursor.execute("INSERT OR IGNORE INTO brands (brand) VALUES (?)", (brand,))

# ✅ Insert Data into Products and Link to brand_id
for _, row in df.iterrows():
    cursor.execute("SELECT brand_id FROM brands WHERE brand = ?", (row["brand"],))
    brand_id = cursor.fetchone()[0]
    
    cursor.execute("""
        INSERT INTO products (brand_id, product_name)
        VALUES (?, ?)
    """, (brand_id, row["Product Name"]))
    
    product_id = cursor.lastrowid  # Get the last inserted product_id
    
    # Insert into specifications
    cursor.execute("""
        INSERT INTO specifications (product_id, battery, display, camera, processor, ram, rom, type)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (product_id, row["Battery"], row["Display"], row["Camera"], row["Processor"], row["RAM"], row["ROM"], row["type"]))
    
    # Insert into ratings_reviews
    cursor.execute("""
        INSERT INTO ratings_reviews (product_id, ratings, reviews)
        VALUES (?, ?, ?)
    """, (product_id, row["Ratings"], row["Reviews"]))

  # Insert into ratings_reviews
    cursor.execute("""
        INSERT INTO price (product_id, price,discount,actual_price)
        VALUES (?, ?, ? ,?)
    """, (product_id, row["Price"], row["Discount"], row["Actual Price"]))

In [14]:
# ✅ Commit and Close
conn.commit()
conn.close()
print(f"✅ Database created successfully at: {db_path}")

✅ Database created successfully at: E:/capstone_data_engeneering/code/android_iphone.db


In [15]:
# ✅ Reconnect and Perform INNER JOIN to Normalize and Reconstruct the Data
conn = sqlite3.connect(db_path)

query = """
SELECT
    b.brand_id, b.brand,
    p.product_id, p.product_name,s.battery, s.display, s.camera, s.processor, s.ram, s.rom, s.type    
FROM brands b
LEFT JOIN products p ON b.brand_id = p.brand_id
LEFT JOIN specifications s ON p.product_id = s.product_id
"""

# ✅ Load the Joined DataFrame
df_specification = pd.read_sql(query, conn)
conn.close()

In [16]:
# ✅ Reconnect and Perform INNER JOIN to Normalize and Reconstruct the Data
conn = sqlite3.connect(db_path)

query = """
SELECT
    b.brand_id, b.brand,
    p.product_id, p.product_name, 
    r.ratings, r.reviews
FROM brands b
LEFT JOIN products p ON b.brand_id = p.brand_id
LEFT JOIN ratings_reviews r ON p.product_id = r.product_id
"""

# ✅ Load the Joined DataFrame
df_ratings = pd.read_sql(query, conn)
conn.close()

In [21]:
query = """
SELECT
    b.brand_id, b.brand,
    p.product_id, p.product_name, 
    r.price, r.discount, r.actual_price
FROM brands b
LEFT JOIN products p ON b.brand_id = p.brand_id
LEFT JOIN price r ON p.product_id = r.product_id
"""
# ✅ Load the Joined DataFrame
df_price = pd.read_sql(query, conn)
conn.close()


In [17]:
df_specification

,brand_id,brand,product_id,product_name,battery,display,camera,processor,ram,rom,type
0,1,vivo,112,"vivo T1 44W (Midnight Galaxy, 128 GB)",5000 mAh Lithium Battery,16.36 cm (6.44 inch) Full HD+ AMOLED Display,50MP + 2MP + 2MP | 16MP Front Camera,Qualcomm Snapdragon 680 Processor,4.0,128.0,Android
1,1,vivo,111,"vivo T1 44W (Starry Sky, 128 GB)",5000 mAh Lithium Battery,16.36 cm (6.44 inch) Full HD+ AMOLED Display,50MP + 2MP + 2MP | 16MP Front Camera,Qualcomm Snapdragon 680 Processor,4.0,128.0,Android
2,1,vivo,132,"vivo T1 5G (Starlight Black, 128 GB)",5000 mAh Lithium Battery,16.71 cm (6.58 inch) Full HD+ Display,50MP + 2MP + 2MP | 16MP Front Camera,Qualcomm Snapdragon 695 Processor,6.0,128.0,Android
3,1,vivo,101,"vivo T1 Pro 5G (Turbo Black, 128 GB)",4700 mAh Lithium Battery,16.36 cm (6.44 inch) Full HD+ AMOLED Display,64MP + 8MP + 2MP | 16MP Front Camera,Qualcomm Snapdragon 778G 5G Mobile Platform Pr...,8.0,128.0,Android
4,1,vivo,114,"vivo T1 Pro 5G (Turbo Cyan, 128 GB)",4700 mAh Lithium Battery,16.36 cm (6.44 inch) Full HD+ AMOLED Display,64MP + 8MP + 2MP | 16MP Front Camera,Qualcomm Snapdragon 778G 5G Mobile Platform Pr...,8.0,128.0,Android
...,...,...,...,...,...,...,...,...,...,...,...
556,5,Apple,526,"Apple iPhone XR (Yellow, 256 GB) (Includes Ear...",A12 Bionic Chip Processor,15.49 cm (6.1 inch) Display,12MP Rear Camera | 7MP Front Camera,A12 Bionic Chip Processor,NaN,256.0,Iphone
557,5,Apple,520,"Apple iPhone XS (Gold, 256 GB)",A12 Bionic Chip Processor,14.73 cm (5.8 inch) Super Retina HD Display,12MP + 12MP | 7MP Front Camera,A12 Bionic Chip Processor,NaN,256.0,Iphone
558,5,Apple,527,"Apple iPhone XS (Gold, 64 GB)",A12 Bionic Chip Processor,14.73 cm (5.8 inch) Super Retina HD Display,12MP + 12MP | 7MP Front Camera,A12 Bionic Chip Processor,NaN,64.0,Iphone
559,5,Apple,560,"Apple iPhone XS (Space Grey, 256 GB)",A12 Bionic Chip Processor,14.73 cm (5.8 inch) Super Retina HD Display,12MP + 12MP | 7MP Front Camera,A12 Bionic Chip Processor,NaN,256.0,Iphone


In [18]:
df_ratings

,brand_id,brand,product_id,product_name,ratings,reviews
0,1,vivo,112,"vivo T1 44W (Midnight Galaxy, 128 GB)",116399.0,7312.0
1,1,vivo,111,"vivo T1 44W (Starry Sky, 128 GB)",116399.0,7312.0
2,1,vivo,132,"vivo T1 5G (Starlight Black, 128 GB)",181273.0,13965.0
3,1,vivo,101,"vivo T1 Pro 5G (Turbo Black, 128 GB)",21239.0,1962.0
4,1,vivo,114,"vivo T1 Pro 5G (Turbo Cyan, 128 GB)",21239.0,1962.0
...,...,...,...,...,...,...
556,5,Apple,526,"Apple iPhone XR (Yellow, 256 GB) (Includes Ear...",100909.0,8480.0
557,5,Apple,520,"Apple iPhone XS (Gold, 256 GB)",11938.0,866.0
558,5,Apple,527,"Apple iPhone XS (Gold, 64 GB)",11938.0,866.0
559,5,Apple,560,"Apple iPhone XS (Space Grey, 256 GB)",11938.0,866.0


In [22]:
df_price

,brand_id,brand,product_id,product_name,price,discount,actual_price
0,1,vivo,112,"vivo T1 44W (Midnight Galaxy, 128 GB)",14499.0,30.0,11999.0
1,1,vivo,111,"vivo T1 44W (Starry Sky, 128 GB)",14499.0,27.0,19990.0
2,1,vivo,132,"vivo T1 5G (Starlight Black, 128 GB)",16990.0,19.0,20990.0
3,1,vivo,101,"vivo T1 Pro 5G (Turbo Black, 128 GB)",21990.0,26.0,18999.0
4,1,vivo,114,"vivo T1 Pro 5G (Turbo Cyan, 128 GB)",24999.0,43.0,16999.0
...,...,...,...,...,...,...,...
556,5,Apple,526,"Apple iPhone XR (Yellow, 256 GB) (Includes Ear...",91900.0,2.0,49900.0
557,5,Apple,520,"Apple iPhone XS (Gold, 256 GB)",103900.0,6.0,79900.0
558,5,Apple,527,"Apple iPhone XS (Gold, 64 GB)",89900.0,8.0,59900.0
559,5,Apple,560,"Apple iPhone XS (Space Grey, 256 GB)",103900.0,2.0,49900.0


In [ ]:
import sqlite3
import pandas as pd

# ✅ Connect to the SQLite database
db_path = "E:/capstone_data_engeneering/code/android_iphone.db"
conn = sqlite3.connect(db_path)


In [ ]:
# ✅ Load tables as DataFrames
df_brands = pd.read_sql("SELECT * FROM brands", conn)
df_products = pd.read_sql("SELECT * FROM products", conn)
df_specifications = pd.read_sql("SELECT * FROM specifications", conn)
df_ratings_reviews = pd.read_sql("SELECT * FROM ratings_reviews", conn)

# ✅ Close the connection
conn.close()


In [ ]:
df_brands.head()

,brand_id,brand
0,1,vivo
1,2,realme
2,3,Samsung
3,4,OPPO
4,5,Apple


In [ ]:
df_products.head()

,product_id,brand_id,product_name,price,discount,actual_price
0,1,1,"vivo T3 Pro 5G (Emerald Green, 256 GB)",24999.0,21.0,31999.0
1,2,1,"vivo T3 Lite 5G (Majestic Black, 128 GB)",11499.0,25.0,15499.0
2,3,1,"vivo T3x 5G (Celestial Green, 128 GB)",13999.0,26.0,18999.0
3,4,1,"vivo V50 5G (Starry Night, 256 GB)",36999.0,13.0,42999.0
4,5,1,"vivo T3 Lite 5G (Vibrant Green, 128 GB)",10499.0,27.0,14499.0


In [ ]:
# ✅ Load tables as DataFrames
df_brands = pd.read_sql("SELECT * FROM brands", conn)
df_products = pd.read_sql("SELECT * FROM products", conn)
df_specifications = pd.read_sql("SELECT * FROM specifications", conn)
df_ratings_reviews = pd.read_sql("SELECT * FROM ratings_reviews", conn)

# ✅ Close the connection
conn.close()

In [ ]:
from lets_plot import *
LetsPlot.setup_html()

ggplot(df_joined, aes(x="ratings", y="price", color="brand")) + \
    geom_point() + ggtitle("Price vs Ratings by Brand")


In [ ]:
df_avg_price = df_joined.groupby("brand")["price"].mean().reset_index()

ggplot(df_avg_price, aes(x="brand", y="price", fill="brand")) + \
    geom_bar(stat="identity") + coord_flip() + ggtitle("Average Price by Brand")


In [ ]:
import plotly.express as px

In [ ]:
fig = px.box(df_joined, x="brand", y="price", color="brand", title="Price Distribution by Brand")
fig.show()
